In [14]:
import sys
sys.path.append("../")

from logic.sms.entities import SingleMachineScheduling, Job, Machine
from logic.sms.cbnb import CombinatorialBnB
from logic.sms.sptf import SPTFRuleScheduler
from logic.sms.completions_sum import WeightedCompletionsSum
from logic.sms.gurobi import GurobiSolver, UpdateValue, SMS_LP_minWeightedSum_timeIndexed

from copy import deepcopy
from time import time
import csv
from func_timeout import func_set_timeout, FunctionTimedOut
import matplotlib.pyplot as plt
import numpy as np
from timeit import timeit
from cProfile import Profile
from pstats import Stats
from pyprof2calltree import convert, visualize

TIMEOUT = 60 * 10    # 10 minutes

def profile(p, objective, solver):
    
    error = None
    elapsed = None
    solverName = type(solver).__name__

    p.objective = objective
    p.solver = solver
    try:
        timeit_timeout = func_set_timeout(TIMEOUT)(timeit)
        elapsed = timeit_timeout("p.solve", globals={"p": p}, number=1)
        if p.value == None or p.value == np.inf or p.value == -np.inf or np.isnan(p.value):
             error = p.value
    except (Exception, FunctionTimedOut) as e:
        error = e
    return elapsed, error, solverName

def profileNJobs(job_sequences, dumpName):
        with open(dumpName + ".csv", "w") as output:
            writer = csv.DictWriter(output, ["solver", "jobs", "elapsed", "error"], lineterminator='\n', delimiter=',')
            writer.writeheader()
            for jobs in job_sequences:
                elapsed, error, solverName = profile(SingleMachineScheduling(deepcopy(jobs), Machine(0)), WeightedCompletionsSum(), CombinatorialBnB(SPTFRuleScheduler))
                writer.writerow({"solver": solverName, "jobs": len(jobs), "elapsed": elapsed, "error": error})
                elapsed, error, solverName = profile(SMS_LP_minWeightedSum_timeIndexed(deepcopy(jobs)), UpdateValue(), GurobiSolver())
                writer.writerow({"solver": solverName, "jobs": len(jobs), "elapsed": elapsed, "error": error})

def plotDataset(dumpName):
       times_gurobi = []
       times_cbnb = []
       n_jobs = []
       
       with open(dumpName + ".csv", "r") as data:
        reader = csv.DictReader(data, delimiter=',')
        for row in reader:
            try:
                elapsed = float(row["elapsed"])
            except ValueError:
                elapsed = np.nan
            solver = row["solver"]
            n = int(row["jobs"])

            if solver == "GurobiSolver":
                times_gurobi.append(elapsed)
            elif solver == "CombinatorialBnB":
                times_cbnb.append(elapsed)
            if n not in n_jobs:
                n_jobs.append(n)

       # plot
       fig, ax = plt.subplots()

       ax.plot(n_jobs, times_gurobi, label="Gurobi")
       ax.plot(n_jobs, times_cbnb, label="CBnB")
       ax.legend()

       plt.show()



def generateJobs(n, p=None, r=None, w=None, rngMethod=None, bounds=(0,1), **kwargs):

    def generateValue(value, rngMethod, default, bounds):

        if value is None and rngMethod is None:
            return default
        elif value is None and rngMethod is not None:
            return rngMethod(**kwargs) * (bounds[1] - bounds[0]) + bounds[0]
        else:
            return value

    
    return [Job(id=j, processingTime=generateValue(p, rngMethod, 1, bounds), releaseTime=generateValue(r, rngMethod, 0, bounds), weight=generateValue(w, rngMethod, 1, bounds)) for j in range(n)]

@func_set_timeout(timeout=TIMEOUT)
def profileSolverCallStack(p, output):
    profileFileName = f"{output}.prof"
    profile = Profile()
    profile.runctx(cmd="p.solve()", globals=globals(), locals={"p":p})
    stats = profile.getstats()
    convert(stats, f"{output}.kgrind")


In [9]:
# BnB vs Gurobi maximum input dimension

N_JOBS = (5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000)
DUMPNAME = f'inputDimension_5-50000_10m'

In [5]:
# dummy test
N_JOBS = (5, 10)
DUMPNAME = f'test'

In [5]:
# BnB vs Gurobi execution times

N_JOBS = (i for i in range (1, 21) for j in range (1000))
DUMPNAME = f'inputDimension_1-20'

In [6]:
job_sequences = [generateJobs(n, p=1, r=0, w=1) for n in N_JOBS]

In [12]:
# Random release time and weights

from numpy.random import default_rng

rng = default_rng(13061998)
job_sequences = [generateJobs(n=20, p=10, rngMethod=rng.uniform, bounds=(0, 50)) for i in range(1000)]
DUMPNAME = "randomProcessReleaseTime_20"

In [13]:
profileNJobs(job_sequences, DUMPNAME)

In [16]:
from numpy.random import default_rng

rng = default_rng(462001)
n = 100
jobs = generateJobs(n, p=10, rngMethod=rng.uniform, bounds=(0, n))

In [15]:
# build problem using bnb scheduler
p = SingleMachineScheduling(jobs, Machine(0))
p.objective = WeightedCompletionsSum()
p.solver = CombinatorialBnB(SPTFRuleScheduler)

# call profiler solver call stack
profileSolverCallStack(p, "bnb_100_callstack")
